# Исследование надёжности заёмщиков
## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
## Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита



## Загрузка библиотек

In [26]:
import pandas as pd
from collections import Counter

#другое
pd.options.display.expand_frame_repr = False
import warnings
warnings.filterwarnings('ignore')

#загрузка файлов с диска
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


## Загрузка и предобработка данных

In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/1_credit_history.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Пропуски в данных

In [3]:
gaps_in_the_data = pd.DataFrame({'percent_of_nulls':df.isnull().sum()/len(df)})
gaps_in_the_data['percent_of_nulls'] = gaps_in_the_data['percent_of_nulls'].map(lambda x: '{:.2%}'.format(x))
print('')
print(df.query('days_employed.isnull()', engine='python')[['days_employed', 'total_income']].sum())
gaps_in_the_data.sort_values(by='percent_of_nulls', ascending=False)


days_employed    0.0
total_income     0.0
dtype: float64


,percent_of_nulls
days_employed,10.10%
total_income,10.10%
children,0.00%
dob_years,0.00%
education,0.00%
education_id,0.00%
family_status,0.00%
family_status_id,0.00%
gender,0.00%
income_type,0.00%


> В двух столбцах пропущены значения, причём они находятся в одних и тех же строках. Один из столбцов с пропущенными значениями - *total_income* — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца *income_type*.

### Обработка пропусков

In [4]:
#найдем среднее значение и медиану дохода и стажа соответственно
total_income_median = df['total_income'].median()
days_employed_mean = df['days_employed'].mean()

df['days_employed'] = df['days_employed'].fillna(days_employed_mean)
df['total_income'] = df['total_income'].fillna(total_income_median)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

In [5]:
#заменим отрицательные числа на положительные в стаже
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(negative_to_pozitive)

> В стаже отрицательные числа сделали положительными, пропуски заменили на среднее значение. В доходе пропуски заменили на медианное значение.
Обратили внимание на неадекватность стажа у пенсионеров, стоят завышенные значения, в 25% выборки. При использовании этих значений в расчетах, мы бы получили неправльную картину. О причинах неадекватности значений надо было бы уточнить у операторов и исправить ошибку.

In [6]:
#проверим формат столбца количества детей
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

> В столбце *children* есть два аномальных значения. -1 заменим на 1, а 20 заменим медианным значением, так как мы не знаем как возникла ошибка: случайно ввели лишний ноль при вводе 2 или же вместо 0 случайно поставили 20

In [7]:
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
pd.DataFrame({'Кол-во заёмщиков по кол-ву детей':df['children'].value_counts()})

,Кол-во заёмщиков по кол-ву детей
0,14225
1,4865
2,2055
3,330
4,41
5,9


### Обработка дубликатов

In [8]:
#приведем в единый формат столбец образование (уберем дубликаты, полученные из-за разного регистра букв)
Counter(df['education'])
df['education'] = df['education'].str.lower()
Counter(df['education'])

Counter({'высшее': 5260,
         'среднее': 15233,
         'неоконченное высшее': 744,
         'начальное': 282,
         'ученая степень': 6})

In [9]:
#проверим формат столбца семейный статус
Counter(df['family_status'])

Counter({'женат / замужем': 12380,
         'гражданский брак': 4177,
         'вдовец / вдова': 960,
         'в разводе': 1195,
         'Не женат / не замужем': 2813})

In [10]:
df.loc[df.loc[:, 'gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


> В столбце *gender* появилось странное значение пола - 'XNA' - скорее всего пропуск, но так как мы не знаем пол человека и не можем сделать предположений из данных, а также в расчетах использовать значение не будем, то оставляем его как есть.

In [11]:
#проверим формат столбца профессии
Counter(df['income_type'])

Counter({'сотрудник': 11119,
         'пенсионер': 3856,
         'компаньон': 5085,
         'госслужащий': 1459,
         'безработный': 2,
         'предприниматель': 2,
         'студент': 1,
         'в декрете': 1})

In [12]:
#проверим формат столбца цель кредита
Counter(df['purpose'])

Counter({'покупка жилья': 647,
         'приобретение автомобиля': 462,
         'дополнительное образование': 462,
         'сыграть свадьбу': 774,
         'операции с жильем': 653,
         'образование': 447,
         'на проведение свадьбы': 777,
         'покупка жилья для семьи': 641,
         'покупка недвижимости': 624,
         'покупка коммерческой недвижимости': 664,
         'покупка жилой недвижимости': 607,
         'строительство собственной недвижимости': 635,
         'недвижимость': 634,
         'строительство недвижимости': 620,
         'на покупку подержанного автомобиля': 479,
         'на покупку своего автомобиля': 505,
         'операции с коммерческой недвижимостью': 651,
         'строительство жилой недвижимости': 626,
         'жилье': 647,
         'операции со своей недвижимостью': 630,
         'автомобили': 478,
         'заняться образованием': 412,
         'сделка с подержанным автомобилем': 489,
         'получение образования': 443,
         'авт

In [13]:
df.duplicated().sum()

71

In [14]:
df = df.drop_duplicates()

### Замена типа данных

In [15]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Категоризация

> Видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Лемматизируем их, чтобы затем разбить на категории. Обе задачи выполним с помощью одной функции.

In [16]:
#столбец цель кредита нужно лемматизировать и на основе его создать новый столбец с категориями

estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']

df['purpose_category'] = 0
def categoryzer(list_of_words, category):
  join = '|'.join(list_of_words)
  index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    df.loc[i, 'purpose_category'] = category
  return df


categoryzer(wedding, 'проведение свадьбы')
categoryzer(estate, 'операции с недвижимостью')
categoryzer(auto, 'операции с автомобилем')
categoryzer(education, 'получение образования')

#проверим результат и убедимся, что значений None не осталось
Counter(df['purpose_category'])

Counter({'операции с недвижимостью': 10811,
         'операции с автомобилем': 4306,
         'получение образования': 4013,
         'проведение свадьбы': 2324})

> В итоге было получено 4 категории целей кредита.

In [17]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children']
debt_from_children

,count_children,sum_children,result_children
children,,,
0,14167,1071,0.075598
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


> Поскольку мы изучаем влияние семейного положения и наличия детей на факт погашения в срок, мы не будем разбивать возраста на категории, так как они нам не понадобятся. Для сдачи проекта категоризация была применена вместе с лемматизацией выше в одной функции. Категоризовать количество детей нет необходимости, категориями будет как раз их количество, так как всего 6 вариантов.

## Исследовательский анализ

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children']
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4855,445,0.091658
3,330,27,0.081818
0,14167,1071,0.075598
5,9,0,0.000000


> С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

### Есть ли зависимость между семейным статусом и возвратом кредита в срок?

In [19]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status']
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


> Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке. Овдовевшие это, как правило, возрастные люди, а они более ответственные. Да и скорее всего у них уже есть взрослые дети, которые могут помогать с выплатами. В целом, да, можно сказать так: самые безответственные те, кто никогда не был в браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Категоризируем заёмщиков в зависимости от их дохода:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [21]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass
df['total_income_category'] = df['total_income'].apply(categorize_income)

По этим категориям и будем оценивать, есть ли зависимость между уровнем дохода заемщика и возвратом кредита в срок.

In [22]:
print('Количество заёмщиков по уровню дохода')
df['total_income_category'].value_counts()

Количество заёмщиков по уровню дохода


C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

In [23]:
debt_from_total_income = pd.DataFrame()
debt_from_total_income['total_income_number'] = df.groupby('total_income_category')['debt'].count()
debt_from_total_income['total_income_sum'] = df.groupby('total_income_category')['debt'].sum()
debt_from_total_income['total_income_mean'] = debt_from_total_income['total_income_sum'] / debt_from_total_income['total_income_number']
debt_from_total_income.sort_values('total_income_mean', ascending=False)

,total_income_number,total_income_sum,total_income_mean
total_income_category,,,
E,22,2,0.090909
C,16016,1360,0.084915
A,25,2,0.080000
B,5041,356,0.070621
D,350,21,0.060000


> Предложенная заказчиком категоризация не самая понятная, попробуем категоризировать доходы заёмщиков по-другому. Пусть доход в каждые 50000 образует новые категории: выполню целое деление дохода 'total_income' на 50000, тем самым разбив заёмщиком на категории каждые 50000 дохода.

In [24]:
def total_income_category(value):
    if value >= 500000:
        return 10
    else:
        value = value // 50000
        return value
df['total_income_id'] = df['total_income'].apply(total_income_category)
df['total_income_id'].value_counts()
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = df.groupby('total_income_id')['debt'].sum()
debt_from_total_income['count'] = df.groupby('total_income_id')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']
debt_from_total_income.sort_values('conversion', ascending = False)

,sum,count,conversion
total_income_id,,,
3,368,4118,0.089364
2,661,7807,0.084668
6,51,624,0.081731
1,331,4091,0.080909
4,164,2254,0.072760
7,24,330,0.072727
8,13,196,0.066327
5,88,1330,0.066165
10,14,222,0.063063


> Ранее мы категоризировали уровень дохода создав доп столбец total_income_id. Мы выполнили целочисленное
деление дохода на 50000, тем самым разбив люей на категории каждые 50000 дохода.
Видим, что начиная с id 10 (зп >= 500000) выборка людей становится нерепрезентативной, поэтому объединяем
людей с доходом => 500000 в одну категорию - id = 10. Для этого добавим в функцию условие.
Очевидно, что зависимости долга от доходов нет, на всякий случай посмотрим корреляцию между столбцами дохода и долгом.

In [25]:
verification_through_correlation = pd.DataFrame()
verification_through_correlation['total_income'] = df['total_income']
verification_through_correlation['debt'] = df['debt']
verification_through_correlation.corr()

,total_income,debt
total_income,1.000000,-0.011797
debt,-0.011797,1.000000


> Зависимости между уровнем дохода и возвратом кредита в срок нет!

### Как разные цели кредита влияют на его возврат в срок?

In [20]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = df.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = df.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category']
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
операции с автомобилем,403,4306,0.093590
получение образования,370,4013,0.092200
проведение свадьбы,186,2324,0.080034
операции с недвижимостью,782,10811,0.072334


> Из последнего столбца видно:
  - чаще всего платят в срок кредиты на недвижимость;
  - чаще просрочка у тех заёмщиков, кому нужен кредит на авто и образование;
  - кредиты взятые на проведение свадьбы заёмщики просрачивают примерно также редко, как кредиты на недвижимость.

## Общий вывод
> Cемейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.